In [1]:
import netCDF4
import numpy as np
import datetime
import sys

In [2]:
def roms_latlon2xy(f,lat,lon,roundvals=True):
    a = abs( f.variables['lat'][:]-lat ) + abs( f.variables['lon'][:]-lon )
    y_coord, x_coord = np.unravel_index(a.argmin(), a.shape)
    if roundvals:
        x_coord = int(np.round(x_coord))
        y_coord = int(np.round(y_coord))
    return x_coord, y_coord

def BFS(start_i, start_j, arr, crit=1):
    """
    Code from josteinb@met.no

    desc:
        Breadth first search function to find index of nearest
        point with crit value (default crit=1 for finding ROMS
        wet-point in mask)
    args:
        - start_i: Start index of i
        - start_j: Start index of j
        - arr: grid with data
        - crit: value to search for (deafult unmasked point)
    return:
        - index of point
    """
    dirs    = [(1,0), (-1,0), (0,1),(0,-1)]
    visited = set()
    q       = [(start_i, start_j)]    # init queue to start pos
    count   = 0
    arrays  = list()  # for storing frames if plotting
    # while something in queue
    while q:
        current = q.pop(0)      # pop the first in waiting queue
        # if we have visited this before
        if current in visited:
            continue
        visited.add(current)    # Add to set of visited
        # If not in border list
        # Test if this is land, if true go to next in queue, else return idx
        if arr[current[0], current[1]] == crit:
            return current[0], current[1]
        count += 1      #updates the count
        # Loop over neighbours and add to queue
        for di, dj in dirs:
            new_i = current[0]+di
            new_j = current[1]+dj
            q.append((new_i, new_j))

In [3]:
filename = 'https://thredds.met.no/thredds/dodsC/sea/norkyst800m/1h/aggregate_be'
nc       = netCDF4.Dataset(filename)
lat      = [60.13035, 59.434547]
lon      = [5.248183, 5.243415]
name     = ['Trollsoy, Austevoll', 'Kvalsvik, Haugesund, Rogaland']
dates    = [datetime.datetime(2021,4,5,12), datetime.datetime(2022,2,11,12)]
depth    = 1 # is at 3 meters, use depth index from [   0.,    3.,   10.,   15.,   25.,   50.,   75.,  100., 150.,  200.,  250.,  300.,  500., 1000., 2000., 3000.]

In [4]:
# Check all arrays length:
if not (len(lat) == len(lon) == len(name) == len(dates)):
    print('Error in array lengths')
    sys.exit(1)

In [5]:
# Loop over all stations:
for i in range(len(name)):
    x, y = roms_latlon2xy(nc, lat[i], lon[i])
    t = np.where(netCDF4.num2date(nc.variables['time'][:], nc.variables['time'].units) == dates[i])[0][0]
    mask = np.where(nc.variables['salinity'][t,depth,:].mask, 0, np.ones_like(nc.variables['salinity'][t,depth,:].mask))
    if mask[y,x] == 0:
        print('Position is on land, will find nearest wet gridpoint')
        y, x = BFS(y,x,mask)
    salt = nc.variables['salinity'][t,depth,y,x]
    temp = nc.variables['temperature'][t,depth,y,x]
    print('model lat {}, lon {}'.format(nc.variables['lat'][y,x], nc.variables['lon'][y,x]))
    print('{}: {}, {} (x, y), salt: {}, temp: {}'.format(name[i], x, y, salt, temp))
    print('-------------------------')

model lat 60.13118388284562, lon 5.25095138261705
Trollsoy, Austevoll: 393, 533 (x, y), salt: 32.59299850463867, temp: 6.699999809265137
-------------------------
Position is on land, will find nearest wet gridpoint
model lat 59.43782925679045, lon 5.231807920605765
Kvalsvik, Haugesund, Rogaland: 305, 493 (x, y), salt: 33.75699996948242, temp: 7.12999963760376
-------------------------
